<a href="https://colab.research.google.com/github/abolfazlshahsavaryyy/DT/blob/main/DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import pandas as pd
import numpy as np


In [118]:
def compute_gini(*p):
    if not (0.999 <= sum(p) <= 1.001):
        raise ValueError("The probabilities must sum to 1.")

    result = 1
    for i in p:
        result -= i**2

    return result

In [119]:
def compute_entropy(*p):
    if not (0.999 <= sum(p) <= 1.001):
        raise ValueError("The probabilities must sum to 1.")

    result = 0
    for i in p:
        if i > 0:
            result += i * np.log2(i)

    return -result

In [120]:
def load_dataset(url:str)->pd.DataFrame:
    raw_df=pd.read_csv(url)
    return raw_df


In [121]:
def dropnan_df(raw_df:pd.DataFrame)->pd.DataFrame:
    raw_df=raw_df.drop("id",axis=1)
    raw_df=raw_df.drop("row",axis=1)
    return raw_df.dropna()


In [122]:
def count_values_columns(df:pd.DataFrame,cols:list):
    for s in cols:
        print(df[s].value_counts())


In [123]:
def Classify_continuous(df:pd.DataFrame,col_name ):

    col=df[col_name]
    min=np.min(col)
    max=np.max(col)
    mean=np.mean([max,min])
    mean_75=np.mean([max,mean])
    mean_25=np.mean([min,mean])
    df.loc[df[col_name]<=min,col_name]=0
    df.loc[(df[col_name]>min )& (df[col_name]<=mean_25),col_name]=1
    df.loc[(df[col_name]>mean_25) & (df[col_name]<=mean),col_name]=2
    df.loc[(df[col_name]>mean) & (df[col_name]<=mean_75),col_name]=3
    df.loc[(df[col_name]>mean_75) & (df[col_name]<=max),col_name]=4
    df.loc[df[col_name]>max ,col_name]=5
    return df

In [124]:
def classify_df(df:pd.DataFrame)-> pd.DataFrame:
    gender_map={
        "Female":0,
        "Male":1
    }
    type_travel_map={
        "Business travel":0,
        "Personal Travel":1
    }
    customer_type_map={
        "Loyal Customer":0,
        "disloyal Customer":1
    }
    class_map={
        "Business":0,
        "Eco":1,
        "Eco Plus":2
    }
    df["Gender"]=df["Gender"].map(gender_map)
    df["Type of Travel"]=df["Type of Travel"].map(type_travel_map)
    df["Customer Type"]=df["Customer Type"].map(customer_type_map)
    df["Class"]=df["Class"].map(class_map)


    return df

In [125]:
url_dataset="Dataset.csv"
raw_df=load_dataset(url_dataset)
raw_df=dropnan_df(raw_df)
df=classify_df(raw_df)
df=Classify_continuous(df,"Age")
df=Classify_continuous(df,"Flight Distance")
df=Classify_continuous(df,"Arrival Delay in Minutes")
df=Classify_continuous(df,"Departure Delay in Minutes")
